<a href="https://colab.research.google.com/github/shivaprajapati34390-netizen/ML-project/blob/main/RAG_System_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers sentence-transformers faiss-cpu langchain langchain-community langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [1]:
from google.colab import files

# This will open a file picker dialog to upload files
uploaded = files.upload()

Saving machine.txt.txt to machine.txt.txt


In [6]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:

# 1. Initialize the Text Splitter
# This splitter is smart. It tries to split on paragraphs ("\n\n"),
# then newlines ("\n"), then spaces (" "), to keep semantically
# related text together as much as possible.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,  # Max size of a chunk
    chunk_overlap=20, # Overlap to maintain context between chunks
    length_function=len
)

In [10]:
# 2. Create the chunks

# Get the content of the uploaded file and decode it from bytes to a string
file_name = 'machine.txt.txt'
file_content = uploaded[file_name].decode('utf-8')

chunks = text_splitter.split_text(file_content)

print(f"We have {len(chunks)} chunks:")
for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i+1} ---\n{chunk}\n")

We have 18284 chunks:
--- Chunk 1 ---
﻿2n
U d

T pd  
e E
n a d
so te it

r d i
F o
lo  fo n

w r
2

--- Chunk 2 ---
w r
2   

Hands-on  
Machine Learning  
 with Scikit-Learn,  
Keras & TensorFlow
Concepts, Tools, and Techniques  
to Build Intelligent Systems

TM

--- Chunk 3 ---
TM

Aurélien Géron






SECOND EDITION

Hands-On Machine Learning with
Scikit-Learn, Keras, and

TensorFlow
Concepts, Tools, and Techniques to

--- Chunk 4 ---
Build Intelligent Systems

Aurélien Géron

Beijing Boston Farnham Sebastopol Tokyo

--- Chunk 5 ---
Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Kiwisoft S.A.S. All rights reserved.

--- Chunk 6 ---
Printed in Canada.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.

--- Chunk 7 ---
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are

--- Chunk 8 ---
also available for most titles (http://oreilly.com). F

In [11]:
# Step 3: Embeddings
# now we turn those text chunks into vectors we use a popular lightweight sentence transformer model.its brillient the understanding meaning of sentence

In [13]:
from sentence_transformers import SentenceTransformer
# 1. Load the embedding model
# 'all-MiniLM-L6-v2' is a fantastic, fast, and small model.
# It runs 100% on your local machine.
model = SentenceTransformer('all-MiniLM-L6-v2')
#  2. Embed all our chunks
# This will take a moment as it "reads" and "understands" each chunk.
chunk_embeddings = model.encode(chunks)

print(f"Shape of our embeddings: {chunk_embeddings.shape}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Shape of our embeddings: (18284, 384)


In [14]:
#  vector store with FAISS
# we have our vector.now we need a database to store them in a way we can search by similarity.it is where faiss comes .its just few lines of code

In [15]:
import faiss
import numpy as np



In [17]:
# get the dimension of our vector
d = chunk_embeddings.shape[1]



In [19]:
#  create the FAISS index
# indexFlatl2 is the simplest most basic index it calculates the exact distance(L2 distance )between our quary and vector.
index=faiss.IndexFlatL2(d)
#  add our chunk into embeddings to the index
# we must convert float32 into FAISS
index.add(np.array(chunk_embeddings).astype(np.float32))

In [22]:
print(f"FAISS index created  with {index.ntotal} vectors")

FAISS index created  with 18284 vectors


In [23]:
# step 5 retrieve,Augment,generate (RAG)
# this is a final part here will user ask aquestion.

In [45]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model for flan-t5-small
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-small')
model_generator = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-small')

Loading weights:   0%|          | 0/190 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [26]:
# this is our RAG pieline function

In [46]:
def answer_question(quary):
  # RETRIEVE
  # embedd the user quary
  quary_embedding=model.encode([quary]).astype('float32')
  # Search the faiss index into top (e.g,k=2) most similar chunk
  k=2
  distance,indices=index.search(quary_embedding,k)
  # get the actual text chunk from our original chunk list
  retrieved_chunks=[chunks[i] for i in indices[0]]
  context="\n\n.".join(retrieved_chunks)
  # 2 Augment
  # this is the magic propmt we combined the retrieved chunks with the uers quary
  prompt_template= f"""
  answer the following question using only the provided context.
  if the answer is not in the context,say \"i don't have any information\"

  Context:
  {context}

  Question:
  {quary}
  Answer:
  """
  # 3 Generate
  # Encode the prompt
  input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids

  # Generate output
  outputs = model_generator.generate(input_ids, max_new_tokens=100, do_sample=False)
  answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(f"---CONTEXT----\n{context}\n")
  return answer

In [39]:
# Example usage of the answer_question function
question_to_ask = "What is machine learning?"
response = answer_question(question_to_ask)
print(f"\n---GENERATED ANSWER---\n{response}")

Passing `generation_config` together with generation-related arguments=({'max_length'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


---CONTEXT----
What Is Machine Learning?                                                                                           2

.What Is Machine Learning?
Machine Learning is the science (and art) of programming computers so they can
learn from data.


---GENERATED ANSWER---

  answer the following question using only the provided context.
  if the answer is not in the context,say " i dont have any information

  Context:
  What Is Machine Learning?                                                                                           2

.What Is Machine Learning?
Machine Learning is the science (and art) of programming computers so they can
learn from data.

  Question:
  What is machine learning?
  Answer:
  


In [47]:
# Example usage of the answer_question function
question_to_ask = "What is machine learning?"
response = answer_question(question_to_ask)
print(f"\n---GENERATED ANSWER---\n{response}")

---CONTEXT----
What Is Machine Learning?                                                                                           2

.What Is Machine Learning?
Machine Learning is the science (and art) of programming computers so they can
learn from data.


---GENERATED ANSWER---
the science (and art) of programming computers so they can learn from data


In [ ]:
print("Hello! I am a chatbot based on your document. Ask me anything about it!")
print("Type 'exit' or 'quit' to end the conversation.")

while True:
    user_question = input("\nYour Question: ")
    if user_question.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break

    response = answer_question(user_question)
    print(f"\nChatbot: {response}")

Hello! I am a chatbot based on your document. Ask me anything about it!
Type 'exit' or 'quit' to end the conversation.

Your Question: what is machine learning
---CONTEXT----
What Is Machine Learning?                                                                                           2

.What Is Machine Learning?
Machine Learning is the science (and art) of programming computers so they can
learn from data.


Chatbot: the science (and art) of programming computers so they can learn from data

Your Question: what is LLM
---CONTEXT----
lem is to use Principal Component Analysis (see Chapter 8), which often results in a
better orientation of the training data.

.m is the number of training instances and n is the number of features); see Table 4-1.


Chatbot: a training data

Your Question: what is large language model
---CONTEXT----
language models, 563 (see also natural language 429

.Recent Innovations in Language Models                                                             